<a href="https://colab.research.google.com/github/Rishika70/Hack_To_Hire/blob/main/Hack_To_Hire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

In [17]:
!pip install huggingface

In [18]:
!pip install transformers

In [19]:
!pip install datasets transformers


In [27]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from transformers import T5Tokenizer, T5Model
from transformers import GPT2Tokenizer, GPT2Model
from nltk.translate.bleu_score import sentence_bleu
from transformers import BertTokenizer, DataCollatorWithPadding
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
import re
import html
from collections import Counter
from transformers import TFBertForSequenceClassification, BertTokenizer, create_optimizer
import tensorflow as tf
from datasets import load_dataset
import torch
from transformers import BertTokenizer




In [21]:
# Tokenize the data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Load Dataset

In [55]:
dataset = load_dataset("toughdata/quora-question-answer-dataset")
df = pd.DataFrame(dataset['train'])  # Convert the dataset to a DataFrame


# Split dataset

In [56]:
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
validation_dataset = train_test_split['test']

# **Analyze the Data**

In [57]:
print(df.head())


                                            question  \
0  Why whenever I get in the shower my girlfriend...   
1            What is a proxy, and how can I use one?   
2  What song has the lyrics "someone left the cak...   
3  I am the owner of an adult website called http...   
4  Does the Bible mention anything about a place ...   

                                              answer  
0  Isn’t it awful? You would swear that there was...  
1  A proxy server is a system or router that prov...  
2                                 MacArthur's Park\n  
3  Don't let apps that are liers put adds on your...  
4  St. John in the book of Revelation mentions an...  


In [58]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56402 entries, 0 to 56401
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  56402 non-null  object
 1   answer    56402 non-null  object
dtypes: object(2)
memory usage: 881.4+ KB
None


In [59]:
print(df.describe())


                                                 question answer
count                                               56402  56402
unique                                               3234  54726
top     Would Hillary Clinton have made a better Presi...   No\n
freq                                                  106     89


In [60]:
print(df.columns)

Index(['question', 'answer'], dtype='object')


# Remove irrelevant information


# **Data Pre-Processing**

# Clean the text , remove urls, Special Characters , stopwords and Lemmatize



In [61]:
def preprocess_text(text):
  # Remove URLs
  text = re.sub(r'http\S+', '', text)

  # Remove special characters and convert to lowercase
  text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()

  # Tokenize the text
  tokens = word_tokenize(text)

  # Remove stop words and lemmatize
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  # Join the tokens back into a string
  return ' '.join(tokens)

# Apply the preprocessing function to the 'question' and 'answer' columns
df['question'] = df['question'].apply(preprocess_text)
df['answer'] = df['answer'].apply(preprocess_text)

Index(['question', 'answer'], dtype='object')


# **Model Evaluation**

# BERT and Metrice Evaluation

In [153]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [154]:

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


# Fine-tune the model on the Quora dataset

In [155]:
import pandas as pd
from transformers import BertTokenizer

# DataFrame with questions and answers
data = {
    "question": ["What is AI?", "How does a car work?"],
    "answer": ["Artificial Intelligence", "Through an internal combustion engine"]
}
df = pd.DataFrame(data)

def preprocess_function(df):
    questions = df["question"].str.lower().tolist()
    answers = df["answer"].str.lower().tolist()

    # Tokenize the questions and answers
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_examples = tokenizer(questions, answers, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    # Convert tensors to lists and add to DataFrame
    df['input_ids'] = tokenized_examples['input_ids'].tolist()
    df['attention_mask'] = tokenized_examples['attention_mask'].tolist()
    df['token_type_ids'] = tokenized_examples['token_type_ids'].tolist()

    return df

# Apply the preprocessing function to the DataFrame
tokenized_df = preprocess_function(df)

print(tokenized_df)


               question                                 answer  \
0           What is AI?                Artificial Intelligence   
1  How does a car work?  Through an internal combustion engine   

                                           input_ids  \
0  [101, 2054, 2003, 9932, 1029, 102, 7976, 4454,...   
1  [101, 2129, 2515, 1037, 2482, 2147, 1029, 102,...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...   

                                      token_type_ids  
0  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, ...  


In [157]:
# For question-answer pairs:
train_encodings = tokenizer(train_df['question'].tolist(), train_df['answer'].tolist(), return_tensors='pt', max_length=512, padding='max_length', truncation=True)
test_encodings = tokenizer(test_df['question'].tolist(), test_df['answer'].tolist(), return_tensors='pt', max_length=512, padding='max_length', truncation=True)

In [159]:
# Apply preprocessing
tokenized_df = preprocess_function(df)

In [160]:
# Define a custom dataset class for our data
class QuoraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Convert labels to numerical representation (assuming labels are strings)
        item['labels'] = torch.tensor(0 if self.labels[idx].lower() == 'no' else 1)  # Example: 'no' -> 0, other -> 1
        return item

    def __len__(self):
        return len(self.labels)

In [161]:
# Create data loaders for training and testing
train_dataset = QuoraDataset(train_encodings, train_df['answer'].tolist())
test_dataset = QuoraDataset(test_encodings, test_df['answer'].tolist())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


# Train The Model

In [167]:
# Define the model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming binary classification

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Access the logits from the model's output
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}: Training Loss = {avg_train_loss:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-160-b8ee25297b90>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1/3: Training Loss = 0.4018
Epoch 2/3: Training Loss = 0.4979
Epoch 3/3: Training Loss = 0.2931


# Model Evaluation

In [168]:
# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

<ipython-input-160-b8ee25297b90>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Accuracy: 1.0
F1 Score: 1.0


# **STATE-OF-THE-ART-LLM**

# GPT Model

In [196]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2PreTrainedModel
import torch
import torch.nn as nn
from torch.optim import Adam

class GPT2ForSequenceClassification(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        logits = self.classifier(hidden_states[:, -1, :])  # Use the last hidden state for classification

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else (logits,)

# Assuming you have a binary classification task (e.g., Question/Answers)
num_labels = 2

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=num_labels)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = Adam(model.parameters(), lr=2e-5)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Data Pre-processing

In [202]:

def preprocess_function(df):
    questions = df["question"].str.lower().tolist()
    answers = df["answer"].str.lower().tolist()

    # Tokenize the questions and answers
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_examples = tokenizer(questions, answers, truncation=True, padding='max_length', max_length=128, return_tensors='pt')  # Adjust max_length as needed

    # Convert tensors to lists and add to DataFrame
    df['input_ids'] = tokenized_examples['input_ids'].tolist()
    df['attention_mask'] = tokenized_examples['attention_mask'].tolist()
    df['token_type_ids'] = tokenized_examples['token_type_ids'].tolist()

    return df

# Apply the preprocessing function to the DataFrame
tokenized_df = preprocess_function(df)

print(tokenized_df)


               question                                 answer  \
0           What is AI?                Artificial Intelligence   
1  How does a car work?  Through an internal combustion engine   

                                           input_ids  \
0  [101, 2054, 2003, 9932, 1029, 102, 7976, 4454,...   
1  [101, 2129, 2515, 1037, 2482, 2147, 1029, 102,...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...   

                                      token_type_ids  
0  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, ...  


In [206]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:  # Use test_loader for evaluation
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs[0]  # Access logits from the tuple returned by GPT2ForSequenceClassification
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


<ipython-input-160-b8ee25297b90>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Accuracy: 0.0
F1 Score: 0.0


In [211]:
predicted_answers = ["This is a predicted answer.", "Another predicted answer."]
reference_answers = [["This is the reference answer."], ["The other reference answer."]]

# Calculate BLEU score
bleu_scores = []
for pred, ref in zip(predicted_answers, reference_answers):
  bleu_score = sentence_bleu(ref, pred)
  bleu_scores.append(bleu_score)

# Print or use the BLEU scores as needed
print(bleu_scores)


[0.5141996115613456, 0.46625954410634507]


# **T5 Model**

In [221]:
# Assuming you have a T5 model and tokenizer
model = T5Model.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Add special tokens to the tokenizer
special_tokens = ['<extra_id_0>', '<extra_id_1>']  # Example special tokens
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# Resize the model's embedding layer to accommodate the new tokens
model.resize_token_embeddings(len(tokenizer))

# Now, when you fine-tune or train the model, the embeddings for the special tokens will be updated along with the rest of the model parameters.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(32100, 512)

In [228]:

def preprocess_function(df):
    questions = df["question"].str.lower().tolist()
    answers = df["answer"].str.lower().tolist()

    # Tokenize the questions and answers
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_examples = tokenizer(questions, answers, truncation=True, padding='max_length', max_length=128, return_tensors='pt')  # Adjust max_length as needed

    # Convert tensors to lists and add to DataFrame
    df['input_ids'] = tokenized_examples['input_ids'].tolist()
    df['attention_mask'] = tokenized_examples['attention_mask'].tolist()
    df['token_type_ids'] = tokenized_examples['token_type_ids'].tolist()

    return df

# Apply the preprocessing function to the DataFrame
tokenized_df = preprocess_function(df)

print(tokenized_df)

               question                                 answer  \
0           What is AI?                Artificial Intelligence   
1  How does a car work?  Through an internal combustion engine   

                                           input_ids  \
0  [101, 2054, 2003, 9932, 1029, 102, 7976, 4454,...   
1  [101, 2129, 2515, 1037, 2482, 2147, 1029, 102,...   

                                      attention_mask  \
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...   
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...   

                                      token_type_ids  
0  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, ...  


In [239]:
# Create data loaders for training and testing
train_dataset = QuoraDataset(train_encodings, train_df['answer'].tolist())
test_dataset = QuoraDataset(test_encodings, test_df['answer'].tolist())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


In [240]:
# Define a custom dataset class for our data
class QuoraDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Convert labels to numerical representation (assuming labels are strings)
        item['labels'] = torch.tensor(0 if self.labels[idx].lower() == 'no' else 1)  # Example: 'no' -> 0, other -> 1
        # Add EOS token if it's not present
        if self.encodings['input_ids'][idx][-1] != tokenizer.eos_token_id:
            item['input_ids'] = torch.cat((item['input_ids'], torch.tensor([tokenizer.eos_token_id])))
            item['attention_mask'] = torch.cat((item['attention_mask'], torch.tensor([1])))
        return item

    def __len__(self):
        return len(self.labels)

In [243]:
# Add special tokens to the tokenizer
special_tokens = ['<extra_id_0>', '<extra_id_1>']  # Example special tokens
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# Resize the model's embedding layer to accommodate the new tokens
model.resize_token_embeddings(len(tokenizer))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def generate_answer(question):
  # Tokenize the input question
  inputs = tokenizer.encode("question: " + question + " </s>", return_tensors="pt").to(device)

  # Generate the answer
  outputs = model.generate(inputs, max_length=100, num_beams=4, early_stopping=True)

  # Decode the generated answer
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return answer

In [244]:
# Example usage:
question = "What is the capital of France?"
generated_answer = generate_answer(question)
print(generated_answer)  # Output: Paris

France


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


# Data Visualization